Run the "environment" code before running any solutions

In [ ]:
#@title environment: google.colab
import re, math
fread = lambda path: from google.colab import files with open(path,'r') as f: f.read()

In [0]:
#@title environment: vscode
import re, math
fread = lambda path: open('inputs/'+path, 'r').read()

In [46]:
#@title Day 1: Trebuchet?!
ls = fread('23-1.txt').splitlines()

print(sum(map(lambda d: int(d[0]+d[-1]),[[c for c in l if '/' < c < ':'] for l in ls])))

ns = {n: n[0]+str(i)+n[-1] for (i,n) in enumerate('zero,one,two,three,four,five,six,seven,eight,nine'.split(','))}
[ls:= [l.replace(k,ns[k]) for l in ls] for k in ns]
print(sum([int(re.search('\\d',l).group()+re.search('\\d',l[::-1]).group()) for l in ls]))


54573
54591


In [0]:
#@title Day 2: Cube Conundrum
maxes = {'red': 12, 'green': 13, 'blue': 14}

ls = fread('23-2.txt').splitlines()
gs = [[{p.strip().split(' ')[1]: int(p.strip().split(' ')[0])
        for p in r.split(',')}
        for r in l.split(':')[1].split(';')]
        for l in ls]

valid = [i+1 for (i,g) in enumerate(gs)
             if all([all([maxes[k]>=r[k] for k in r]) for r in g])]
print(sum(valid))

powers = [math.prod([max([r.get(k,0) for r in g]) for k in maxes]) for g in gs]
print(sum(powers))

2727
56580


In [0]:
#@title Day 3-1: Gear Ratios
f = fread('23-3.txt').splitlines()

special = lambda x,y,n: [re.search('[^\\d.]',f[min(max(yi,0),len(f))][max(x-1,0):min(x+n+1,len(f[0]))]) != None for yi in range(y-1,y+2)]
ps = {(m.start(),y): m.group() for (y,l) in enumerate(f) for m in re.finditer('\\d+',l)}
ns = [int(ps[(x,y)]) for (x,y) in ps if any(
     [re.search('\\d+',f[min(max(yi,0),len(f)-1)][max(x-1,0):min(x+len(ps[(x,y)])+1,len(f[0])-1)]) != None for yi in range(y-1,y+2)]
     )]
print(sum(ns))

529618


In [91]:
#@title Day 4: Scratchcards 
f = fread('23-4.txt').splitlines()

# cs = [[set(map(int, [n for n in ss.strip().split(' ') if n != '']))
#   for ss in re.split('(\\|||(Card \\d+:))', l) if ss != None]
#   for l in f]
cs = [[set([int(n) for n in m.group().split(' ') if n != '']) 
  for m in re.finditer('(\\d+ +)+', l)] 
  for l in f]

ws = [sum([1 for n in c[1] if c[0].contains(c[1])])
  for c in cs]

print(cs)

AttributeError: 'set' object has no attribute 'contains'